In [1]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
import numpy as np
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from imbd.trainers import ModelTrainer
from imbd.data import DataLoader
from imbd.preprocessors import DataPreprocessor

In [2]:
loader = DataLoader()
prepro = DataPreprocessor()

def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(20))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def create_func_model():
    inputs = tf.keras.Input(shape=(293,))
    x = tf.keras.layers.Dense(64)(inputs)
    outputs = tf.keras.layers.Dense(20)(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


In [3]:
    param_grid = {
        "prepro__variance_selector__threshold": [0.0],
        # "model__estimator__n_estimators": [1000],
        # "model__estimator__max_depth": [5, 10],
        # "model__estimator__alpha": [0, 0.1, 0.01],
        # "model__estimator__lambda": [1, 0.5, 0.1],
        # "model__estimator__subsample": [1, 0.5],
        # "model__estimator__gamma": [0, 2, 10],
    }

In [4]:
# initialization
loader = DataLoader()
preprocessor = DataPreprocessor()
df = loader.build()

# get feature & label
train_features = df.drop(loader.labels, axis=1)
train_labels = df[loader.labels]


In [5]:
base_model = MultiOutputRegressor(XGBRegressor())
# base_nn_model = KerasRegressor(build_fn=create_model, epochs=100)
base_nn_model = KerasRegressor(build_fn=create_model, epochs=100)

In [6]:
# build pipeline
steps = [('prepro', preprocessor), ('model', base_nn_model)]
pipe = Pipeline(steps=steps)

In [7]:
# training
trainer = ModelTrainer(pipe=pipe, param_grid=param_grid, verbose=2)
fitted = trainer.train(train_features, train_labels)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] prepro__variance_selector__threshold=0.0 ........................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
292
Epoch 1/100
[CV] ......... prepro__variance_selector__threshold=0.0, total=   1.1s
[CV] prepro__variance_selector__threshold=0.0 ........................
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
289
Epoch 1/100
[CV] ......... prepro__variance_selector__threshold=0.0, total=   0.8s
[CV] prepro__variance_selector__threshold=0.0 ........................
293
Epoch 1/100
8/8 [==============================] - 0s 6ms/step - loss: 8218.0391
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 1823.0842
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 343.2648
Epoch 4/100
8/8 [==============================] - 0s 2ms/step - loss: 113.7823
Epoch 5/100
8/8 [==============================] - 0s 2ms/step - lo

In [8]:
fitted.best_score_

nan

In [9]:
np.abs(fitted.predict(train_features) - train_labels)

293


,Input_A6_024,Input_A3_016,Input_C_013,Input_A2_016,Input_A3_017,Input_C_050,Input_A6_001,Input_C_096,Input_A3_018,Input_A6_019,Input_A1_020,Input_A6_011,Input_A3_015,Input_C_046,Input_C_049,Input_A2_024,Input_C_058,Input_C_057,Input_A3_013,Input_A2_017
0,0.286742,0.486058,1.061660,0.062364,0.661478,0.531679,0.151108,0.062715,0.057925,0.441707,0.071889,0.433032,1.261627,0.584437,0.331451,0.612257,0.387300,0.408022,0.545370,0.136589
1,0.202472,0.222659,0.102260,0.178699,0.120206,0.522276,0.761442,0.454674,0.907476,0.048870,0.241779,0.152193,0.044849,0.477749,0.685401,0.162297,0.040892,0.161624,0.235434,0.111094
2,0.041830,0.286463,0.166358,0.269153,0.387779,0.502470,0.187407,0.400345,0.201411,0.231370,0.240200,0.516925,0.469276,0.313096,0.124536,0.044421,0.885170,0.380450,0.174234,0.008393
3,0.501441,0.860935,0.812607,0.181665,0.489082,0.548579,0.352739,0.648485,0.782033,1.138806,0.520106,0.301844,0.122653,0.631043,0.505454,0.620362,0.853913,0.628278,0.010393,0.039976
4,0.402066,0.288642,0.243648,0.388051,0.527425,0.788948,0.041971,0.251888,0.427016,0.245842,0.369337,0.073092,0.035672,0.345505,0.051996,0.081450,0.009500,0.575735,0.445435,0.040300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,0.221643,0.137322,0.415535,0.287241,0.174811,0.083198,0.832583,0.294103,0.264600,0.216215,0.492516,0.315594,0.291219,0.578705,0.350224,0.350926,0.506324,0.148171,0.046526,0.056868
335,1.014708,0.546206,0.264109,0.030659,0.210615,0.074433,0.013914,0.897492,0.106254,0.381597,0.503782,0.205373,0.082455,0.004882,0.329083,1.168812,0.055876,0.174809,0.199637,0.125589
336,0.444172,0.271113,0.015033,0.111857,0.426393,0.033244,0.360036,0.916466,0.668931,0.152731,0.860954,0.209770,0.148713,0.292803,0.420948,0.427907,0.056493,0.138351,0.597641,0.247192
337,0.107421,0.371932,0.629629,0.589626,0.270849,0.044645,0.225328,0.062849,0.192133,0.298835,0.769998,0.572180,0.596455,0.274310,0.671780,0.624567,0.053178,0.225962,0.735758,0.392469
